In [43]:
# import packages 
import pandas as pd
import numpy as np
import matplotlib
import glob
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

In [2]:
def load_kickstarter_data(datapath):
    '''datapath = location of csv files to be loaded'''
    # List with the names of all the csv files in the path
    csv_files = glob.glob(datapath+'/*.csv')

    print(f'Total files: {len(csv_files)}')

    # Loop through the files
    for file_idx, csv_file in enumerate(csv_files): 
        # create dataframe from 1st csv       
        if file_idx == 0:
            df_ks = pd.read_csv(csv_file)
            print(f'File number {file_idx + 1} added to dataframe')
        else:
            # create dataframe from idx csv
            df = pd.read_csv(csv_file)
            # check files are all in same
            if  np.all(df.columns == df_ks.columns) == False:
                print(f'Column format of {csv_file} does not match {csv_files[0]}. Please check and try again')
                return
            else:
                # append to initial dataframe                   
                df_ks = pd.concat([df_ks, df], axis=0, ignore_index=True)       
                print(f'File number {file_idx + 1} added to dataframe')
    print('File import done')
    return df_ks

In [3]:
def load_kickstarter_data_short(datapath):
    ''' 
    This is a version of the main function to load jsut 2 files for use in testing
    datapath = location of csv files to be loaded
    '''
    # List with the names of all the csv files in the path
    csv_files = glob.glob(datapath+'/*.csv')

    print(f'Total files: {len(csv_files)}')

    # Loop through the files
    for file_idx, csv_file in enumerate(csv_files): 
        # create dataframe from 1st csv       
        if file_idx == 0:
            df_ks = pd.read_csv(csv_file)
            print(f'File number {file_idx + 1} added to dataframe')
        else:
            # create dataframe from idx csv
            df = pd.read_csv(csv_file)
            # check files are all in same
            if  np.all(df.columns == df_ks.columns) == False:
                print(f'Column format of {csv_file} does not match {csv_files[0]}. Please check and try again')
                return
            else:
                # append to initial dataframe                   
                df_ks = pd.concat([df_ks, df], axis=0, ignore_index=True)       
                print(f'File number {file_idx + 1} added to dataframe')
                # This is here to prevent more than 2 files being loaded to save time in testing
                break
    print('File import done')
    return df_ks

In [4]:
df = load_kickstarter_data_short('kickstarter/data')
#df.state.value_counts()

Total files: 56
File number 1 added to dataframe
File number 2 added to dataframe
File import done


In [38]:
def extract_json_data(data):
    ''' This function extracts specific sub fields from json files embedded in columns of a dataframe
        data: dataframe containing column with json data'''
    data['category_slug'] = pd.DataFrame([json.loads(data["category"][i])['slug'] for i in range(data.shape[0])])
    data['category_name'] = pd.DataFrame.from_dict([json.loads(data["category"][i])['name'] for i in range(data.shape[0])])
    data = data.drop('category', axis=1, inplace=True)
    
# other Jsons are not parsable by this method
#    data['location_name'] = pd.DataFrame.from_dict([json.loads(data["location"][i])['name'] for i in range(data.shape[0])])
#    data['location_country'] = pd.DataFrame([json.loads(data["location"][i])['country'] for i in range(data.shape[0])])
#    data['creater_is_registered'] = pd.DataFrame([json.loads(data["creator"][i])['is_registered'] for i in range(data.shape[0])])
    print('json columns extracted')
    return data
# check output
#df2 = extract_json_data(df)
#df2.head()

In [6]:
#features_raw

In [20]:
def get_target_features(data):
    '''
    Function that splits dataset into target and feature dataframes
    '''
    target = data['state']
    features_raw = data.drop('state', axis = 1)
    print('target and features split is done')
    return target, features_raw

In [8]:
def scale_features(data, num_columns):
    ''' Initialize a scaler, then apply it to the features'''
    scaler = MinMaxScaler()
    data[num_columns] = scaler.fit_transform(data[num_columns])
    return data

In [9]:
def test_train_split_kickstarter(features, target, test_size = 0.2, random_state = 0):
    '''
    Split data into train and test sets based on features and target dataframes.
    Shows results of the split and returns four dataframes
    '''
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = test_size, random_state = random_state)
    # Show the results of the split
    print ("Training set has {} samples.".format(X_train.shape[0]))
    print ("Testing set has {} samples.".format(X_test.shape[0]))
    return X_train, X_test, y_train, y_test

In [10]:
'''# this doesn't work

#data['category_slug'] = pd.DataFrame([json.loads(data["category"][i])["slug"] for i in range(data.shape[0])])
json_columns = ['category'] #, 'creator', 'urls', 'location']
for col in json_columns:
    for i in range(len(df[col])):
        try:
            dict_cat = json.loads(df[col][i])
        except:
            dict_cat = 0
    df_json = pd.DataFrame.from_dict(dict_cat) 
    df = pd.merge(left=df, right=df_json, left_index=True, right_index=True, suffixes = ('',col) )      
df_json.head()
#    

#df0.head()'''

'# this doesn\'t work\n\n#data[\'category_slug\'] = pd.DataFrame([json.loads(data["category"][i])["slug"] for i in range(data.shape[0])])\njson_columns = [\'category\'] #, \'creator\', \'urls\', \'location\']\nfor col in json_columns:\n    for i in range(len(df[col])):\n        try:\n            dict_cat = json.loads(df[col][i])\n        except:\n            dict_cat = 0\n    df_json = pd.DataFrame.from_dict(dict_cat) \n    df = pd.merge(left=df, right=df_json, left_index=True, right_index=True, suffixes = (\'\',col) )      \ndf_json.head()\n#    \n\n#df0.head()'

In [11]:
def train_using_gini(X_train, X_test, y_train): 
  
    # Creating the classifier object 
    clf_gini = DecisionTreeClassifier(criterion = "gini", 
            max_depth=3, min_samples_leaf=5) 
    # Performing training 
    clf_gini.fit(X_train, y_train) 
    return clf_gini 

In [12]:
# Function to make predictions 
def prediction(X_test, clf_object): 
  
    # Predicton on test data with model trained using either giniIndex / or entropy
    y_pred = clf_object.predict(X_test) 
    print("Predicted values:\n") 
    print(y_pred) 
    return y_pred 

In [13]:
def cal_accuracy(y_test, y_pred): 
      
    print("Confusion Matrix: \n", 
        confusion_matrix(y_test, y_pred)) 
      
    print ("Accuracy : \n", 
    accuracy_score(y_test,y_pred)*100) 
      
    print("Report : \n", 
    classification_report(y_test, y_pred)) 

In [ ]:
""" MAIN SCRIPT ============================================================="""
if __name__ == '__main__':
    
    # Read data
    df = load_kickstarter_data_short('kickstarter/data')
    # drop campaigns that are still ongoing
    df = df[df.state != 'live']
    
    # Split the data into features and target label
    target, features_raw = get_target_features(df)
    

    # encode target variable 'state' to numerical values, success is 1 all others are fail and 0
    target = target.apply(lambda x: 1 if x == 'success' else 0)

    # split categorical columns into dummies
#    features_raw = pd.get_dummies(features_raw)    

    # Clean and augment data
    # extract data from json columns
    features_raw = extract_json_data(features_raw)
    # scale numerical features
    num_columns = ['goal', 'campaign_length']
    features_raw = scale_features(df, num_columns)
    
    # Split into training and test set
    X_train, X_test, y_train, y_test = test_train_split_kickstarter(features_raw, target, test_size = 0.2, random_state = 0)
    
    # Fit a simple decision tree first
    clf_gini = train_using_gini(X_train, X_test, y_train) 
    
    # Create predictions using simple model
    y_pred = prediction(X_test, clf_gini)
    
    # show results
    cal_accuracy(y_test, y_pred)
    
    # Creating the classifier object 


    # Fit Adaboost classifier using a decision tree as base estimator
    # Test with different number of iterations
    # Append the score output from each iteration of the adboost function
    
    # Compare error rate vs number of iterations
    #   plot_error_rate(er_train, er_test)

In [ ]:
df.slug

In [17]:
df.columns

Index(['backers_count', 'blurb', 'category', 'converted_pledged_amount',
       'country', 'created_at', 'creator', 'currency', 'currency_symbol',
       'currency_trailing_code', 'current_currency', 'deadline',
       'disable_communication', 'friends', 'fx_rate', 'goal', 'id',
       'is_backing', 'is_starrable', 'is_starred', 'launched_at', 'location',
       'name', 'permissions', 'photo', 'pledged', 'profile', 'slug',
       'source_url', 'spotlight', 'staff_pick', 'state', 'state_changed_at',
       'static_usd_rate', 'urls', 'usd_pledged', 'usd_type', 'category_slug',
       'category_name'],
      dtype='object')

In [39]:
df = load_kickstarter_data_short('kickstarter/data')
    # drop campaigns that are still ongoing

Total files: 56
File number 1 added to dataframe
File number 2 added to dataframe
File import done


In [45]:
df.goal.value_counts()

5000.0      520
10000.0     470
1000.0      378
2000.0      300
15000.0     289
           ... 
10550.0       1
164200.0      1
51000.0       1
1374.0        1
2695.0        1
Name: goal, Length: 671, dtype: int64

In [ ]:
#df = df[df.state != 'live']
    
    # Split the data into features and target label

In [15]:
df3 = extract_json_data(df)

json columns extracted


In [22]:
target, features_raw = get_target_features(df)

target and features split is done


In [23]:
features_raw.head()

,backers_count,blurb,category,converted_pledged_amount,country,created_at,creator,currency,currency_symbol,currency_trailing_code,...,profile,slug,source_url,spotlight,staff_pick,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
0,315,Babalus Shoes,"{""id"":266,""name"":""Footwear"",""slug"":""fashion/fo...",28645,US,1541459205,"{""id"":2094277840,""name"":""Lucy Conroy"",""slug"":""...",USD,$,True,...,"{""id"":3508024,""project_id"":3508024,""state"":""in...",babalus-childrens-shoes,https://www.kickstarter.com/discover/categorie...,False,False,1548223375,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",28645.000000,international
1,47,A colorful Dia de los Muertos themed oracle de...,"{""id"":273,""name"":""Playing Cards"",""slug"":""games...",1950,US,1501684093,"{""id"":723886115,""name"":""Lisa Vollrath"",""slug"":...",USD,$,True,...,"{""id"":3094785,""project_id"":3094785,""state"":""ac...",the-ofrenda-oracle-deck,https://www.kickstarter.com/discover/categorie...,True,False,1504976459,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",1950.000000,domestic
2,271,"Electra's long awaited, eclectic Debut Pop/Roc...","{""id"":43,""name"":""Rock"",""slug"":""music/rock"",""po...",22404,US,1348987533,"{""id"":323849677,""name"":""Electra"",""is_registere...",USD,$,True,...,"{""id"":359847,""project_id"":359847,""state"":""inac...",record-electras-debut-album-pop-rock-classical,https://www.kickstarter.com/discover/categorie...,True,False,1371013395,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",22404.000000,international
3,3,The Mist of Tribunal is a turn-based card game...,"{""id"":273,""name"":""Playing Cards"",""slug"":""games...",165,GB,1483780271,"{""id"":196281496,""name"":""Artur Ordijanc (delete...",GBP,£,False,...,"{""id"":2825329,""project_id"":2825329,""state"":""in...",the-mist-of-tribunal-a-card-game,https://www.kickstarter.com/discover/categorie...,False,False,1489425776,1.216066,"{""web"":{""project"":""https://www.kickstarter.com...",165.384934,domestic
4,3,"Livng with a brain impairment, what its like t...","{""id"":48,""name"":""Nonfiction"",""slug"":""publishin...",2820,US,1354817071,"{""id"":1178460181,""name"":""Dawn Johnston"",""is_re...",USD,$,True,...,"{""id"":417385,""project_id"":417385,""state"":""inac...",help-change-the-face-of-brain-impairment,https://www.kickstarter.com/discover/categorie...,True,False,1357763527,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",2820.000000,domestic


In [24]:
    # encode target variable 'state' to numerical values, success is 1 all others are fail and 0
target = target.apply(lambda x: 1 if x == 'success' else 0)

In [ ]:
    # split categorical columns into dummies
#    features_raw = pd.get_dummies(features_raw)    

    # Clean and augment data

In [25]:

features_raw = extract_json_data(features_raw)


json columns extracted


In [28]:
num_columns = ['goal']
features_raw = scale_features(df, num_columns)

In [30]:
# Split into training and test set
X_train, X_test, y_train, y_test = test_train_split_kickstarter(features_raw, target, test_size = 0.2, random_state = 0)

Training set has 6057 samples.
Testing set has 1515 samples.


In [37]:
X_train.shape

(6057, 37)

In [31]:
# Fit a simple decision tree first
clf_gini = train_using_gini(X_train, X_test, y_train) 

ValueError: could not convert string to float: 'A picture book about a dragon raised by a trickster rabbit who rescues him.'

In [27]:
# Create predictions using simple model
y_pred = prediction(X_test, clf_gini)

# show results
cal_accuracy(y_test, y_pred)

# Creating the classifier object 


# Fit Adaboost classifier using a decision tree as base estimator
# Test with different number of iterations
# Append the score output from each iteration of the adboost function

# Compare error rate vs number of iterations
#   plot_error_rate(er_train, er_test)

Training set has 6057 samples.
Testing set has 1515 samples.


ValueError: could not convert string to float: 'A picture book about a dragon raised by a trickster rabbit who rescues him.'